In [1]:
#load libraries and the classification model

import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
import shutil
from PIL import Image
from tensorflow.keras.models import load_model
model = load_model('avo_model.keras')

C:\Users\johnl\AppData\Local\Temp\ipykernel_19940\3686910131.py:6: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr
c:\Users\johnl\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 16 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


First we run the yolo model to identify the kind of avo. Since we can only predict Hass, maybe I need something to account for this. 

Next we move these images into a separate checking folder. This is to make the rest of the code sustainable and work across all devices consistently. This will move all the Hass crops into a separate checking folder (or shepards at a later date, may need an if else or something) and then delete the runs folder to avoid having the future cops go into 'predict2' and then having to keep track of the number of predicts.

In [2]:
!yolo predict model=avoID.pt source='Pictures/1020' conf = 0.5 save_crop=True save
shutil.move('runs/detect/predict/crops/Hass','checking_folder/') #need to add a line if there are no hass' detected
shutil.rmtree('runs/')

Ultralytics 8.3.91 🚀 Python-3.12.9 torch-2.6.0+cpu CPU (AMD Ryzen 7 3700X 8-Core Processor)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs

image 1/7 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1020\482736535_936074375403619_6568392030990003422_n.jpg: 640x512 1 Hass, 121.3ms
image 2/7 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1020\483134351_9435740509878282_1840051949765373307_n.jpg: 640x512 1 Hass, 99.3ms
image 3/7 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1020\488607736_3071586109682685_3680888725088702396_n.jpg: 640x512 1 Hass, 98.7ms
image 4/7 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1020\488850548_969574118298263_4386745660247402047_n.jpg: 640x512 2 Hasss, 99.6ms
image 5/7 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1020\488921841_585538767871724_5559439902032879049_n.jpg: 640x512 2 Hasss, 97.8ms
image 6/7 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1020\490996281_9470889019696806_664628569

In [3]:
#This is to resize the image prior to running it through the model
def prepare_input(image_path, target_size=(256, 256)): #adjust target size to match model input.
    img = Image.open(image_path).resize(target_size)
    img_array = np.array(img) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0) # Add batch dimension
    return img_array

avo_pics = os.listdir('checking_folder/')

avo_score = [] #empty list to store the avo score

#Loop through all the pics in the folder to get the individual predictions for each avo
for pic in avo_pics:
    avo = prepare_input('checking_folder/' + pic)
    avo_score.append(model.predict(avo))

shutil.rmtree('checking_folder/') #delete the checking_folder to ensure we can use exactly the same code in the futur

avo_score = np.concatenate(avo_score, axis=0 )
avo_score.mean()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


3.4497638